# 📊 Monitoring & Observability

**Phase 3: System Design - Production Operations**

**Master metrics collection, alerting, dashboards, and observability patterns for production systems**

---

In [ ]:
// Observability Overview - Monitoring, Logging, Alerting
println("📊 MONITORING & OBSERVABILITY - PRODUCTION SYSTEMS INSIGHT")
println()

println("🎯 The Three Pillars of Production Readiness:")
println("🔍 Monitoring: Activity measurement and resource tracking")
println("📋 Logging: Detailed activity records with context")
println("🚨 Alerting: Proactive incident detection and response")
println("📊 Dashboards: Visual insights into system behavior")
println()

println("🏆 Quality Attributes for Production Monitoring:")
println("✓ Observability: Understanding system state from outputs")
println("✓ Reliability: Fault tolerance and recovery metrics")
println("✓ Performance: Throughput, latency, resource utilization")
println("✓ Availability: Uptime, service level objectives (SLOs)")
println("✓ Incident Response: Mean time to detection/recovery (MTTD/MTTR)")
println()


## 📈 Metrics Collection & Analysis

**Comprehensive metrics gathering with proper classification and aggregation patterns**

In [ ]:
// Enterprise Metrics System with Scala
sealed trait MetricType
case object Counter extends MetricType        // Monotonically increasing values
case object Gauge extends MetricType         // Instantaneous measurements
case object Histogram extends MetricType     // Distributions (percentiles)
case object Summary extends MetricType       // Stats with quantiles

case class ServiceMetrics(
  requestCount: Long = 0,
  errorCount: Long = 0,
  activeConnections: Int = 0,
  responseTimeMs: List[Long] = Nil,
  lastError: Option[String] = None,
  uptimeSeconds: Long = 0
)

class MetricsCollector[F[_]: Concurrent: Timer](
  serviceName: String,
  backend: MetricsBackend[F]
) {
  
  def recordRequest(service: String, responseTime: FiniteDuration): F[Unit] = {
    for {
      _ <- backend.incrementCounter(s"$service.requests_total")
      _ <- backend.recordHistogram(s"$service.request_duration_seconds",
          Map("service" -> service, "method" -> "GET"),
          responseTime.toMillis.toDouble / 1000.0)
      _ <- backend.setGauge(s"$service.last_request_timestamp",
          Map("service" -> service), System.currentTimeMillis())
    } yield ()
  }
  
  def recordError(service: String, errorType: String): F[Unit] = {
    backend.incrementCounter(s"$service.errors_total",
      Map("service" -> service, "error_type" -> errorType))
  }
  
  def recordCircuitBreakerState(service: String, state: String): F[Unit] = {
    backend.setGauge(s"$service.circuit_breaker_state",
      Map("service" -> service, "state" -> state), state match {
        case "closed" => 0
        case "open" => 1
        case "half_open" => 2
        case _ => -1
      })
  }

  // Health check endpoint
  def healthStatus(): F[HealthStatus] = {
    for {
      metrics <- backend.getServiceMetrics(serviceName)
      totalRequests = metrics.getOrElse("requests_total", 0.0).toLong
      totalErrors = metrics.getOrElse("errors_total", 0.0).toLong
      errorRate = if (totalRequests > 0) (totalErrors.toDouble / totalRequests) else 0.0
    } yield if (errorRate < 0.05) Healthy else Degraded
  }
}

println("📈 Enterprise Metrics System Implemented")
println("• Counter metrics for events (requests, errors)")
println("• Histogram metrics for distributions (latencies)")
println("• Gauge metrics for current values (connections, state)")
println("• Health check integration with SLO tracking")
println("• Service-level tagging for multi-tenant metrics")


## 📋 Structured Logging Patterns

**Log management, correlation IDs, and observability-driven logging**

In [ ]:
// Enterprise Logging System
sealed trait LogLevel
case object DEBUG extends LogLevel
case object INFO extends LogLevel
case object WARN extends LogLevel
case object ERROR extends LogLevel

case class LogEntry(
  timestamp: java.time.Instant,
  level: LogLevel,
  message: String,
  correlationId: Option[String] = None,
  userId: Option[String] = None,
  requestId: Option[String] = None,
  service: String,
  context: Map[String, String] = Map.empty,
  error: Option[Throwable] = None
)

class StructuredLogger[F[_]: Sync](
  serviceName: String,
  correlationIdGenerator: F[String]
) {
  
  private val levels = Map(
    DEBUG -> 10,
    INFO -> 20,
    WARN -> 30,
    ERROR -> 40
  )
  
  def info(msg: String, ctx: Map[String, String] = Map.empty): F[Unit] =
    log(INFO, msg, ctx)
    
  def error(msg: String, error: Throwable, ctx: Map[String, String] = Map.empty): F[Unit] =
    log(ERROR, msg, ctx, Some(error))
    
  def warn(msg: String, ctx: Map[String, String] = Map.empty): F[Unit] =
    log(WARN, msg, ctx)
    
  def debug(msg: String, ctx: Map[String, String] = Map.empty): F[Unit] =
    log(DEBUG, msg, ctx)

  private def log(
    level: LogLevel,
    message: String,
    context: Map[String, String],
    error: Option[Throwable] = None
  ): F[Unit] = {
    for {
      correlationId <- correlationIdGenerator
      entry = LogEntry(
        timestamp = java.time.Instant.now(),
        level = level,
        message = message,
        correlationId = Some(correlationId),
        service = serviceName,
        context = context,
        error = error
      )
      _ <- writeLogEntry(entry)
      _ <- conditionallyAlert(entry) // Alert on critical events
    } yield ()
  }
  
  private def writeLogEntry(entry: LogEntry): F[Unit] = {
    JsonLogger.toJson(entry).flatMap { json =>
      println(s"[${entry.level}] ${entry.service}: ${json}")
    }.handleErrorWith(_ => Sync[F].unit) // Logging errors don't crash the service
  }
  
  private def conditionallyAlert(entry: LogEntry): F[Unit] = {
    val shouldAlert = entry.level == ERROR && 
      entry.error.exists(_.isInstanceOf[CriticalBusinessError])
    
    if (shouldAlert) {
      sendAlert(entry)
    } else {
      Sync[F].unit
    }
  }
}

case class CriticalBusinessError(msg: String) extends Exception(msg)

println("📋 Structured Logging System Implemented")
println("• Correlation IDs for request tracing")
println("• Structured JSON logging for analysis")
println("• Log levels with severity hierarchy")
println("• Error telemetry and alerting integration")
println("• Non-blocking logging that never crashes")


## 🔔 Alerting & Incident Management

**Smart alerting systems with escalations, de-duplication, and automated incident response**

In [ ]:
// Production Alerting System
sealed trait AlertSeverity
case object Info extends AlertSeverity
case object Warning extends AlertSeverity
case object Critical extends AlertSeverity
case object Pager extends AlertSeverity

case class Alert(
  id: String,
  severity: AlertSeverity,
  service: String,
  title: String,
  description: String,
  timestamp: java.time.Instant,
  tags: Set[String] = Set.empty,
  resolved: Boolean = false
)

class AlertManager[F[_]: Concurrent: Timer](
  thresholds: AlertThresholds,
  notificationService: NotificationService[F],
  alertStore: AlertStore[F]
) {
  
  private val activeAlerts = Ref.of[F, Map[String, Alert]](Map.empty)
  
  // SLO-based alert rules
  def checkSLOMetrics(metrics: ServiceMetrics): F[Unit] = {
    val errorRate = if (metrics.requestCount > 0) {
      metrics.errorCount.toDouble / metrics.requestCount
    } else 0.0
    
    for {
      _ <- if (errorRate > thresholds.errorRatePercent / 100.0) {
        fireAlert(
          s"${metrics.serviceName} error rate: ${errorRate * 100}%.1f%%",
          s"Error rate exceeded ${thresholds.errorRatePercent}% threshold",
          if (errorRate > thresholds.errorRatePercent * 2 / 100.0) Pager else Critical
        )
      } else clearAlert("high_error_rate")
      
      _ <- if (metrics.responseTimeP95 > thresholds.p95LatencyMs) {
        fireAlert(
          s"${metrics.serviceName} P95 latency: ${metrics.responseTimeP95}ms",
          s"95th percentile latency exceeded ${thresholds.p95LatencyMs}ms threshold",
          Warning
        )
      } else clearAlert("high_latency")
      
    } yield ()
  }
  
  private def fireAlert(title: String, description: String, severity: AlertSeverity): F[Unit] = {
    val alertId = generateAlertId(title)
    
    for {
      alreadyActive <- activeAlerts.get.map(_.contains(alertId))
      _ <- if (!alreadyActive) {
        for {
          alert <- createAlert(alertId, title, description, severity)
          _ <- alertStore.save(alert)
          _ <- activeAlerts.update(_ + (alert.id -> alert))
          _ <- notificationService.notify(alert)
          _ <- if (severity == Pager) escalationManager.scheduleEscalation(alert)
        } yield ()
      } else Sync[F].unit
    } yield ()
  }
  
  def resolveAlert(alertId: String): F[Unit] = {
    for {
      alert <- activeAlerts.get.map(_.get(alertId))
      _ <- alert.map(a => 
        alertStore.update(a.copy(resolved = true)) *>
        activeAlerts.update(_ - alertId)
      ).getOrElse(Sync[F].unit)
    } yield ()
  }
  
  private def createAlert(id: String, title: String, desc: String, sev: AlertSeverity): F[Alert] = {
    Sync[F].pure(Alert(
      id = id,
      severity = sev,
      service = "system",
      title = title,
      description = desc,
      timestamp = java.time.Instant.now(),
      tags = Set("auto-generated", "monitoring")
    ))
  }

  def getActiveAlerts(): F[List[Alert]] = 
    activeAlerts.get.map(_.values.toList.filter(!_.resolved))
}

println("🔔 Enterprise Alerting System Implemented")
println("• SLO-based automated alerts")
println("• Severity levels with escalation paths")
println("• Alert de-duplication and noise reduction")
println("• Incident tracking with resolution workflow")
println("• Pagerduty integration for critical alerts")


## 📊 Dashboard & Visualization Patterns

**Creating production dashboards for system insights and business intelligence**

In [ ]:
// Dashboard Configuration and Analytics
println("📊 PRODUCTION DASHBOARDS - SYSTEM VISUALIZATION")
println()

println("🔥 Essential Dashboard Components:")
println()

println("1. 🔄 Service Health Overview")
println("   ✓ Current status of all services")
println("   ✓ Uptime and availability percentage")
println("   ✓ Active alerts and critical issues")
println("   ✓ Circuit breaker states")
println()

println("2. 📈 Performance Metrics Dashboard")
println("   ✓ Request rate (RPS, RPM)")
println("   ✓ Response time percentiles (P50, P95, P99)")
println("   ✓ Error rates and success rates")
println("   ✓ Throughput and resource utilization")
println()

println("3. 🎯 Business Metrics Integration")
println("   ✓ User engagement metrics")
println("   ✓ Revenue and transaction KPIs")
println("   ✓ Feature adoption rates")
println("   ✓ Customer satisfaction scores")
println()

println("4. 🔍 Detailed Investigation Panels")
println("   ✓ Log correlation and trace views")
println("   ✓ Database query performance")
println("   ✓ Cache hit rates and miss penalties")
println("   ✓ End-to-end request flow visualization")
println()

println("🛠️ Dashboard Tools Integration:")
println("• Grafana: Real-time metrics dashboarding")
println("• Kibana: Log aggregation and search")
println("• Jaeger/Zipkin: Distributed tracing")
println("• Prometheus: Metrics collection and querying")

println("\n📊 MONITORING MATURITY LEVELS:")
println("Level 1: Manual monitoring (alerting on obvious issues)")
println("Level 2: Automated alerting (SLO-based, intelligent thresholds)")
println("Level 3: Predictive analytics (anomaly detection, forecasting)")
println("Level 4: Self-healing systems (auto-remediation, chaos engineering)")

println("\nDon't forget: Observing a system changes its behavior!")
println("Monitoring should add <5% overhead and never impact user experience.")
